# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [ ]:
#pip install geopandas
#pip install fastparquet

In [1]:
# all import statements needed for the project, for example:

import math
from math import sin, cos, sqrt, atan2, radians
import bs4
from bs4 import BeautifulSoup
import json
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
import geopandas as gpd
import numpy as np
import re
import fastparquet

In [ ]:
# any general notebook setup, like log formatting

In [2]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

### Calculating distance
_**TODO:** Write some prose that tells the reader what you're about to do here._

In [3]:
def calculate_distance(df):
# turning all coordinates into radians because all trig functions are in radians not degrees 
    lat1 = []
    for coord in df["pickup_longitude"]: 
        lat1.append(radians(coord))
    
    lon1 = []
    for coord in df["pickup_latitude"]: 
        lon1.append(radians(coord))

    lat2 = []
    for coord in df["dropoff_longitude"]: 
        lat2.append(radians(coord))
    
    lon2 = []
    for coord in df["dropoff_latitude"]: 
        lon2.append(radians(coord))
        
    
# calculate difference in coordinates 
    diflon = list()
    for item1, item2 in zip(lon1, lon2): 
        diflon.append(item2 - item1)

    diflat = list()
    for item1, item2 in zip(lat1, lat2): 
        diflat.append(item2 - item1)  
        
        
# calculating distance using formula 
    R = 6373.0
    dist = list()
    for rad1, rad2, item1, item2 in zip(diflon, diflat, lat1, lat2):
        a = sin(rad2 / 2)**2 + cos(item1) * cos(item2) * sin(rad1 / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c 
        dist.append(distance)
        
    return dist

In [4]:
def add_distance_column(df):
    df["distance"] = calculate_distance(df)
    return df 

### Processing Uber Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [5]:
def load_and_clean_uber_data(csv_file):
    # import/read the csv file 
    UBER_DATA = pd.read_csv(csv_file) 
    
    # dropping unnecesarry columns
    # UBER_DATA = UBER_DATA.drop(UBER_DATA.iloc[:, 0:2],axis = 1) 
    UBER_DATA = UBER_DATA[["pickup_datetime", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"]]
    
    # dropping rows where coordinates == 0
    # create a boolean mask to check if values in rows are == 0
    # use ~ to invert boolean mask
    UBER_DATA = UBER_DATA[~(UBER_DATA[['pickup_longitude','pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']] == 0).any(axis=1)] 
    
    # defining longitude and latitude bounds 
    # longitude 
    westlimit = -74.242330
    eastlimit = -73.717047
    # latitude 
    southlimit = 40.560445
    northlimit = 40.908524
    
    # filtering out pickup data 
    pickup_longitude = UBER_DATA['pickup_longitude'].values
    pickup_latitude = UBER_DATA['pickup_latitude'].values
    
        # create filters for data within longitude limits 
    pickupwest = pickup_longitude >= westlimit
    pickupeast = pickup_longitude <= eastlimit 
    pickuplon = pickupwest * pickupeast 
    
        # create filters for data within latitude limits 
    pickupnorth = pickup_latitude <= northlimit 
    pickupsouth = pickup_latitude >= southlimit
    pickuplat = pickupnorth * pickupsouth 
    
    
        # create final pickup filter 
    
    pickupfilter = pickuplon * pickuplat
        

    # filtering out dropoff data
    dropoff_longitude = UBER_DATA['dropoff_longitude'].values
    dropoff_latitude = UBER_DATA['dropoff_latitude'].values
    
    
    # create filters for data within longitude limits 
    dropoffwest = dropoff_longitude >= westlimit
    dropoffeast = dropoff_longitude <= eastlimit 
    dropofflon = dropoffwest * dropoffeast 
    
        # create filters for data within latitude limits 
    dropoffnorth = dropoff_latitude <= northlimit 
    dropoffsouth = dropoff_latitude >= southlimit
    dropofflat = dropoffnorth * dropoffsouth 
    
    
        # create final dropoff filter 
    dropofffilter = dropofflon * dropofflat
    
    
    # final boundary filter
    finalfilter = pickupfilter * dropofffilter
    
    # final filtered dataframe
    UBER_DATA = UBER_DATA[finalfilter]
    
    
    # normalize column datatypes 
    UBER_DATA["pickup_datetime"]=pd.to_datetime(UBER_DATA["pickup_datetime"])



    return UBER_DATA
    


In [6]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_CSV)
    add_distance_column(uber_dataframe)
    return uber_dataframe

In [7]:
get_uber_data()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,distance
0,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,0.465327
1,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,0.678941
2,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,4.825036
3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,1.262035
4,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5.371588
...,...,...,...,...,...,...
199995,2012-10-28 10:49:00+00:00,-73.987042,40.739367,-73.986525,40.740297,0.064197
199996,2014-03-14 01:09:00+00:00,-73.984722,40.736837,-74.006672,40.739620,2.442986
199997,2009-06-29 00:42:00+00:00,-73.986017,40.756487,-73.858957,40.692588,14.269270
199998,2015-05-20 14:56:25+00:00,-73.997124,40.725452,-73.983215,40.695415,1.800660


### Processing Taxi Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [8]:
def find_taxi_csv_urls():
    response = requests.get(TAXI_URL)
    response_page = BeautifulSoup(response.content, "lxml")
    elements = response_page.find_all('a', href=True)


    list1 = []
    for ele in elements: 
        list1.append(ele['href'])
    
    links = []
    for i in range(455): 
        string = str(list1[i])
        pattern = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_20\d\d-\d\d.parquet"
        if re.search(pattern, string) is not None: 
            links.append(string)

    links = links[80:]
    
    # putting links in chronological order 
    year15 = links[:6] # through June of 2015
    year14 = links[12:24]
    year13 = links[24:36]
    year12 = links[36:48]
    year11 = links[48:60]
    year10 = links[60:72]
    year09 = links[72:]
    
    final_links = year09 + year10 + year11 + year12 + year13 + year14 + year15
   
    return final_links

In [9]:
def download_taxi_data(links):
    # downloading the parquet files using the links
    for link in links:
        filename = link[48:]
        response = requests.get(link)
        open(filename,'wb').write(response.content)

In [ ]:
taxi_urls = find_taxi_csv_urls()
download_taxi_data(taxi_urls)

In [10]:
def get_and_clean_month_taxi_data1(url):
    
    pattern1 = "yellow_tripdata_20\d\d-\d\d.parquet"
    if re.search(pattern1, url) is not None: 
        filename = re.search(pattern1, url).group()
    
    dataset = pd.read_parquet(filename, engine='fastparquet')
    
    dataset = dataset[["Trip_Pickup_DateTime", "Start_Lon", "Start_Lat", "End_Lon", "End_Lat"]]
    dataset = dataset.set_axis(["pickup_datetime", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"], axis=1, inplace=False)
    df1 = dataset
    # dropping rows where coordinates == 0
    # create a boolean mask to check if values in rows are == 0
    # use ~ to invert boolean mask
    dataset = dataset[~(dataset[['pickup_longitude','pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']] == 0).any(axis=1)] 
    
    # defining longitude and latitude bounds 
    # longitude 
    westlimit = -74.242330
    eastlimit = -73.717047
    # latitude 
    southlimit = 40.560445
    northlimit = 40.908524
    
    # filtering out pickup data 
    tpickup_longitude = dataset['pickup_longitude'].values
    tpickup_latitude = dataset['pickup_latitude'].values
    
        # create filters for data within longitude limits 
    tpickupwest = tpickup_longitude >= westlimit
    tpickupeast = tpickup_longitude <= eastlimit 
    tpickuplon = tpickupwest * tpickupeast 
    
        # create filters for data within latitude limits 
    tpickupnorth = tpickup_latitude <= northlimit 
    tpickupsouth = tpickup_latitude >= southlimit
    tpickuplat = tpickupnorth * tpickupsouth 
    
    
        # create final pickup filter 
    
    tpickupfilter = tpickuplon * tpickuplat
        

    # filtering out dropoff data
    tdropoff_longitude = dataset['dropoff_longitude'].values
    tdropoff_latitude = dataset['dropoff_latitude'].values
    
        
        # create filters for data within longitude limits 
    tdropoffwest = tdropoff_longitude >= westlimit
    tdropoffeast = tdropoff_longitude <= eastlimit 
    tdropofflon = tdropoffwest * tdropoffeast 
    
        # create filters for data within latitude limits 
    tdropoffnorth = tdropoff_latitude <= northlimit 
    tdropoffsouth = tdropoff_latitude >= southlimit
    tdropofflat = tdropoffnorth * tdropoffsouth 
    
    
        # create final dropoff filter 
    tdropofffilter = tdropofflon * tdropofflat
    
    
    # final boundary filter
    tfinalfilter = tpickupfilter * tdropofffilter
    
    # final filtered dataframe
    dataset = dataset[tfinalfilter]
    
    # create a random sample of data 
    # ~200,000 (size of uber sample)
    # 84 (number of taxi datasets)
    # ~ 2380 sampled rows per taxi dataset
    dataset = dataset.sample(2380)
    
    
    # normalize column datatypes 
    dataset["pickup_datetime"]=pd.to_datetime(dataset["pickup_datetime"])

    
    return dataset

In [11]:
def get_and_clean_month_taxi_data2(url):
    
    pattern1 = "yellow_tripdata_20\d\d-\d\d.parquet"
    if re.search(pattern1, url) is not None: 
        filename = re.search(pattern1, url).group()
    
    dataset = pd.read_parquet(filename, engine='fastparquet')
    
    dataset = dataset[["pickup_datetime", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"]]
    

    # dropping rows where coordinates == 0
    # create a boolean mask to check if values in rows are == 0
    # use ~ to invert boolean mask
    dataset = dataset[~(dataset[['pickup_longitude','pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']] == 0).any(axis=1)] 
    
    # defining longitude and latitude bounds 
    # longitude 
    westlimit = -74.242330
    eastlimit = -73.717047
    # latitude 
    southlimit = 40.560445
    northlimit = 40.908524
    
    # filtering out pickup data 
    tpickup_longitude = dataset['pickup_longitude'].values
    tpickup_latitude = dataset['pickup_latitude'].values
    
        # create filters for data within longitude limits 
    tpickupwest = tpickup_longitude >= westlimit
    tpickupeast = tpickup_longitude <= eastlimit 
    tpickuplon = tpickupwest * tpickupeast 
    
        # create filters for data within latitude limits 
    tpickupnorth = tpickup_latitude <= northlimit 
    tpickupsouth = tpickup_latitude >= southlimit
    tpickuplat = tpickupnorth * tpickupsouth 
    
    
        # create final pickup filter 
    
    tpickupfilter = tpickuplon * tpickuplat
        

    # filtering out dropoff data
    tdropoff_longitude = dataset['dropoff_longitude'].values
    tdropoff_latitude = dataset['dropoff_latitude'].values
    
        
        # create filters for data within longitude limits 
    tdropoffwest = tdropoff_longitude >= westlimit
    tdropoffeast = tdropoff_longitude <= eastlimit 
    tdropofflon = tdropoffwest * tdropoffeast 
    
        # create filters for data within latitude limits 
    tdropoffnorth = tdropoff_latitude <= northlimit 
    tdropoffsouth = tdropoff_latitude >= southlimit
    tdropofflat = tdropoffnorth * tdropoffsouth 
    
    
        # create final dropoff filter 
    tdropofffilter = tdropofflon * tdropofflat
    
    
    # final boundary filter
    tfinalfilter = tpickupfilter * tdropofffilter
    
    # final filtered dataframe
    dataset = dataset[tfinalfilter]
    
    # create a random sample of data 
    # ~200,000 (size of uber sample)
    # 84 (number of taxi datasets)
    # ~ 2380 sampled rows per taxi dataset
    dataset = dataset.sample(2380)
    
    
    # normalize column datatypes 
    dataset["pickup_datetime"]=pd.to_datetime(dataset["pickup_datetime"])

    
    return dataset

In [12]:
def get_and_clean_month_taxi_data3(url):
    
    shapefile = gpd.read_file("taxi_zones.shp")
    shapefile = shapefile[["LocationID","geometry"]]
    shapefile = shapefile.to_crs(epsg = 4326)
    shapefile["lon"] = shapefile.geometry.centroid.x
    shapefile["lat"] = shapefile.geometry.centroid.y
    shapefile.index = np.arange(1,264)
    shapefile = shapefile[['lon','lat']]
    longitude = shapefile['lon']
    latitude = shapefile['lat']
    
    pattern = "yellow_tripdata_20\d\d-\d\d.parquet"
    if re.search(pattern, url) is not None: 
        filename = re.search(pattern, url).group()
    
    dataset = pd.read_parquet(filename, engine='fastparquet')
    dataset = dataset[["tpep_pickup_datetime", "PULocationID", "DOLocationID", "trip_distance"]]
    dataset = dataset.set_axis(["pickup_datetime", "PULocationID", "DOLocationID", "distance"], axis=1, inplace=False)
    
    pulocationidfilter = dataset['PULocationID'] <= 263
    dolocationidfilter = dataset['DOLocationID'] <= 263
    validityfilter = pulocationidfilter & dolocationidfilter
    
    dataset = dataset[validityfilter] # gets rid  of nonvalid IDs 

    pulon = []
    pulat = []
    for idpu in dataset['PULocationID']: 
        pulon.append(longitude[idpu])
        pulat.append(latitude[idpu])
        
    dataset['pickup_longitude'] = pulon
    dataset['pickup_latitude'] = pulat
        
    dolon = []
    dolat = []
    for iddo in dataset['DOLocationID']: 
        dolon.append(longitude[iddo])
        dolat.append(latitude[iddo])
        
    dataset['dropoff_longitude'] = dolon
    dataset['dropoff_latitude'] = dolat
    
    dataset = dataset[["pickup_datetime", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"]]
    
        # dropping rows where coordinates == 0
    # create a boolean mask to check if values in rows are == 0
    # use ~ to invert boolean mask
    dataset = dataset[~(dataset[['pickup_longitude','pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']] == 0).any(axis=1)] 
    
    # defining longitude and latitude bounds 
    # longitude 
    westlimit = -74.242330
    eastlimit = -73.717047
    # latitude 
    southlimit = 40.560445
    northlimit = 40.908524
    
    # filtering out pickup data 
    tpickup_longitude = dataset['pickup_longitude'].values
    tpickup_latitude = dataset['pickup_latitude'].values
    
        # create filters for data within longitude limits 
    tpickupwest = tpickup_longitude >= westlimit
    tpickupeast = tpickup_longitude <= eastlimit 
    tpickuplon = tpickupwest * tpickupeast 
    
        # create filters for data within latitude limits 
    tpickupnorth = tpickup_latitude <= northlimit 
    tpickupsouth = tpickup_latitude >= southlimit
    tpickuplat = tpickupnorth * tpickupsouth 
    
    
        # create final pickup filter 
    
    tpickupfilter = tpickuplon * tpickuplat
        

    # filtering out dropoff data
    tdropoff_longitude = dataset['dropoff_longitude'].values
    tdropoff_latitude = dataset['dropoff_latitude'].values
    
        
        # create filters for data within longitude limits 
    tdropoffwest = tdropoff_longitude >= westlimit
    tdropoffeast = tdropoff_longitude <= eastlimit 
    tdropofflon = tdropoffwest * tdropoffeast 
    
        # create filters for data within latitude limits 
    tdropoffnorth = tdropoff_latitude <= northlimit 
    tdropoffsouth = tdropoff_latitude >= southlimit
    tdropofflat = tdropoffnorth * tdropoffsouth 
    
    
        # create final dropoff filter 
    tdropofffilter = tdropofflon * tdropofflat
    
    
    # final boundary filter
    tfinalfilter = tpickupfilter * tdropofffilter
    
    # final filtered dataframe
    dataset = dataset[tfinalfilter]
    
    # create a random sample of data 
    # ~200,000 (size of uber sample)
    # 84 (number of taxi datasets)
    # ~ 2380 sampled rows per taxi dataset
    dataset = dataset.sample(2380)
    
    
    # normalize column datatypes 
    dataset["pickup_datetime"]=pd.to_datetime(dataset["pickup_datetime"])
              
    return dataset
  

In [13]:
def get_and_clean_taxi_data():
    
    all_taxi_dataframes = []
    taxi_urls = find_taxi_csv_urls()
    csv_urls1 = taxi_urls[:12]
    csv_urls2 = taxi_urls[12:24]
    csv_urls3 = taxi_urls[24:]

    
    for csv_url in csv_urls1:
        dataframe1 = get_and_clean_month_taxi_data1(csv_url)
        add_distance_column(dataframe1)
        all_taxi_dataframes.append(dataframe1)
    
    for csv_url in csv_urls2:
        dataframe2 = get_and_clean_month_taxi_data2(csv_url)
        add_distance_column(dataframe2)
        all_taxi_dataframes.append(dataframe2)
        
        
    for csv_url in csv_urls3:
        dataframe3 = get_and_clean_month_taxi_data3(csv_url)
        add_distance_column(dataframe3)
        all_taxi_dataframes.append(dataframe3)
        
    # create one gigantic dataframe with data from every month needed
    #taxi_data = pd.concact(all_taxi_dataframes)
    
    
    taxi_data = []
    for dataframe in all_taxi_dataframes: 
        taxi_data.append(dataframe)
        
                            
    return taxi_data

In [14]:
taxi_urls = find_taxi_csv_urls()
#get_and_clean_month_taxi_data1(taxi_urls[11])
#get_and_clean_month_taxi_data2(taxi_urls[12])

def test(url): 
    pattern = "yellow_tripdata_20\d\d-\d\d.parquet"
    if re.search(pattern, url) is not None: 
        filename = re.search(pattern, url).group()
    
    dataset = pd.read_parquet(filename, engine='fastparquet')
    
    return dataset

In [ ]:
#taxi_data = get_and_clean_taxi_data()

### Processing Weather Data

_In this section, the processesing of data before performing analyses continues. The data is split into two different dataframes so that alayses can be run at different frequencies. NaN and invalid data points are dropped as part of cleaning the data. Regex is also used to extract just didgits from the dataframe. _

In [15]:
def clean_month_weather_data_hourly(csv_file):
    
    # import/read csv file 
    data = pd.read_csv(csv_file, low_memory = False) 
    # drop NAN values for precipitation
    cleaned_data = data.dropna(axis=0, subset = "HourlyPrecipitation") 
    # dropping rows where precipitation == T (trace amounts)
    # create a boolean mask to check if values in rows are == T
    # use ~ to invert boolean mask
    cleaned_data = cleaned_data[~(cleaned_data["HourlyPrecipitation"] == 'T')]
    # only extract digits from column 
    cleaned_data["HourlyPrecipitation"] = cleaned_data["HourlyPrecipitation"].replace('\(|[a-zA-Z]+', '', regex=True)
    # drop NAN values for wind speed
    cleaned_data = cleaned_data.dropna(axis=0, subset = "HourlyWindSpeed") 
    # create a dataframe with only the necessary columns 
    cleaned_data = cleaned_data[["DATE","HourlyPrecipitation","HourlyWindSpeed"]]
    # standardize datatypes
    cleaned_data["DATE"] = pd.to_datetime(cleaned_data["DATE"])
    cleaned_data["HourlyPrecipitation"] = pd.to_numeric(cleaned_data["HourlyPrecipitation"])
    
    return cleaned_data

In [16]:
def clean_month_weather_data_daily(csv_file):
    # load cleaned hourly data
    cleaned_hourly = clean_month_weather_data_hourly(csv_file)
    # groupby each day and assign the average of the hourly values to a daily value
    daily = cleaned_hourly.groupby([cleaned_hourly['DATE'].dt.date]).mean()
    
    return daily

In [17]:
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    # add the name/paths manually
    weather_csv_files = ["2009_weather.csv", "2010_weather.csv","2011_weather.csv","2012_weather.csv","2013_weather.csv","2014_weather.csv","2015_weather.csv" ]
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
    # create two dataframes with hourly & daily data from every month (YEAR?)
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

### Process All Data

_This is where you can actually execute all the required functions._

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [26]:
taxi_data = get_and_clean_taxi_data()

C:\Users\johnf\AppData\Local\Temp\ipykernel_20196\705598918.py:10: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  dataset = dataset.set_axis(["pickup_datetime", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"], axis=1, inplace=False)
C:\Users\johnf\AppData\Local\Temp\ipykernel_20196\705598918.py:10: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  dataset = dataset.set_axis(["pickup_datetime", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"], axis=1, inplace=False)
C:\Users\johnf\AppData\Local\Temp\ipykernel_20196\705598918.py:10: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  dataset = dataset.se

In [27]:
taxi_data = pd.concat(taxi_data)
taxi_data

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,distance
6891767,2009-01-08 22:35:00,-73.981912,40.773760,-73.932862,40.797493,5.504381
14030472,2009-01-16 10:22:30,-73.955361,40.768600,-73.953716,40.764845,0.216348
10841276,2009-01-19 17:00:00,-73.953937,40.766288,-73.970498,40.788372,1.963107
10888292,2009-01-22 23:34:00,-74.000023,40.732717,-73.986537,40.732678,1.500046
3420072,2009-01-31 13:32:00,-73.966833,40.788605,-73.967702,40.761842,0.827826
...,...,...,...,...,...,...
9038939,2015-06-22 19:13:16,-73.978492,40.747746,-73.965635,40.768615,1.567144
179167,2015-06-01 13:11:57,-73.976495,40.740439,-73.984052,40.736824,0.847901
2894518,2015-06-07 18:49:26,-73.990458,40.740337,-73.981414,40.670374,2.370846
11367055,2015-06-28 10:34:30,-73.965635,40.768615,-73.951010,40.778766,1.656349


In [24]:
uber_data = get_uber_data()
uber_data

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,distance
0,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,0.465327
1,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,0.678941
2,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,4.825036
3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,1.262035
4,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5.371588
...,...,...,...,...,...,...
199995,2012-10-28 10:49:00+00:00,-73.987042,40.739367,-73.986525,40.740297,0.064197
199996,2014-03-14 01:09:00+00:00,-73.984722,40.736837,-74.006672,40.739620,2.442986
199997,2009-06-29 00:42:00+00:00,-73.986017,40.756487,-73.858957,40.692588,14.269270
199998,2015-05-20 14:56:25+00:00,-73.997124,40.725452,-73.983215,40.695415,1.800660


In [18]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()
daily_weather_data = daily_weather_data.rename(columns={'HourlyPrecipitation':'DailyPrecipitation',
                                  'HourlyWindSpeed':'DailyWindSpeed'})
daily_weather_data

C:\Users\johnf\AppData\Local\Temp\ipykernel_20196\1294668373.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  daily = cleaned_hourly.groupby([cleaned_hourly['DATE'].dt.date]).mean()
C:\Users\johnf\AppData\Local\Temp\ipykernel_20196\1294668373.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  daily = cleaned_hourly.groupby([cleaned_hourly['DATE'].dt.date]).mean()
C:\Users\johnf\AppData\Local\Temp\ipykernel_20196\1294668373.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or 

,DailyPrecipitation,DailyWindSpeed
DATE,,
2009-01-06,0.017143,8.857143
2009-01-07,0.058710,10.387097
2009-01-10,0.020500,9.250000
2009-01-11,0.039231,8.769231
2009-01-15,0.010000,7.000000
...,...,...
2015-12-27,0.007391,5.521739
2015-12-28,0.001500,8.150000
2015-12-29,0.028182,7.303030


## Part 2: Storing Cleaned Data

While the dataframes we have stored our data in are convenient for coding purposes, they are impermanent and take a long time to process, so we would like to store them so that we do not have to generate them every time. We do this by writing them into SQL tables in a database we define. We will name our database "project.db" and write our dataframes into it by using Pandas' .to_sql functionality.

In [19]:
engine = db.create_engine('sqlite:///project.db')

In [76]:
hourly_weather_data.to_sql('hourlyWeather',con=engine, if_exists='replace',index_label='id')
daily_weather_data.to_sql('dailyWeather',con=engine, if_exists='replace',index_label='id')
uber_data.to_sql('uber',con=engine, if_exists='replace',index_label='id')
taxi_data.to_sql('taxi',con=engine, if_exists='replace',index_label='id')

185640

## Part 3: Understanding the Data

In this section, we will run several queries on the data we have stored in SQL tables. We do this to get a better understanding of the shape and scope of our data. For each query, we have a particular question in mind that we will answer by calling relevant data. At times, we may summarize the data by counting rows or taking averages. In the next cell, we define a function that will write the prose of our queries into .sql files so that we can call them later without having to write them again.

In [98]:
def write_query_to_file(query, outfile):
    """Write a given SQL query string to a file called outfile."""
    f = open(outfile,'w')
    f.write(query)
    f.close()

### Query 1

Here we will find what hour of the day was the most popular to take a Yellow Taxi by counting the number of rows in our table corresponding to each hour of the day.

In [28]:
QUERY_1 = """
SELECT 
COUNT(CASE WHEN pickup_datetime LIKE '%00:%' THEN 1 END) AS '00',
COUNT(CASE WHEN pickup_datetime LIKE '%01:%' THEN 1 END) AS '01',
COUNT(CASE WHEN pickup_datetime LIKE '%02:%' THEN 1 END) AS '02',
COUNT(CASE WHEN pickup_datetime LIKE '%03:%' THEN 1 END) AS '03',
COUNT(CASE WHEN pickup_datetime LIKE '%04:%' THEN 1 END) AS '04',
COUNT(CASE WHEN pickup_datetime LIKE '%05:%' THEN 1 END) AS '05',
COUNT(CASE WHEN pickup_datetime LIKE '%06:%' THEN 1 END) AS '06',
COUNT(CASE WHEN pickup_datetime LIKE '%07:%' THEN 1 END) AS '07',
COUNT(CASE WHEN pickup_datetime LIKE '%08:%' THEN 1 END) AS '08',
COUNT(CASE WHEN pickup_datetime LIKE '%09:%' THEN 1 END) AS '09',
COUNT(CASE WHEN pickup_datetime LIKE '%10:%' THEN 1 END) AS '10',
COUNT(CASE WHEN pickup_datetime LIKE '%11:%' THEN 1 END) AS '11',
COUNT(CASE WHEN pickup_datetime LIKE '%12:%' THEN 1 END) AS '12',
COUNT(CASE WHEN pickup_datetime LIKE '%13:%' THEN 1 END) AS '13',
COUNT(CASE WHEN pickup_datetime LIKE '%14:%' THEN 1 END) AS '14',
COUNT(CASE WHEN pickup_datetime LIKE '%15:%' THEN 1 END) AS '15',
COUNT(CASE WHEN pickup_datetime LIKE '%16:%' THEN 1 END) AS '16',
COUNT(CASE WHEN pickup_datetime LIKE '%17:%' THEN 1 END) AS '17',
COUNT(CASE WHEN pickup_datetime LIKE '%18:%' THEN 1 END) AS '18',
COUNT(CASE WHEN pickup_datetime LIKE '%19:%' THEN 1 END) AS '19',
COUNT(CASE WHEN pickup_datetime LIKE '%20:%' THEN 1 END) AS '20',
COUNT(CASE WHEN pickup_datetime LIKE '%21:%' THEN 1 END) AS '21',
COUNT(CASE WHEN pickup_datetime LIKE '%22:%' THEN 1 END) AS '22',
COUNT(CASE WHEN pickup_datetime LIKE '%23:%' THEN 1 END) AS '23'
FROM taxi
"""

In [29]:
engine.execute(QUERY_1).fetchall()

[(10198, 8420, 7064, 5899, 5128, 4950, 6710, 9844, 11445, 11797, 11223, 11812, 12021, 12016, 12171, 11885, 10501, 12413, 13956, 14770, 13902, 13585, 13149, 11934)]

In [30]:
write_query_to_file(QUERY_1, "popular_taxi_hours.sql")

### Query 2

Here we will find what day was the most popular to take an Uber by counting the number of rows in our table corresponding to each day of the week.


In [31]:
QUERY_2 = """
SELECT
COUNT(CASE WHEN strftime('%w',pickup_datetime) IS '0' THEN 1 END) AS 'Sunday',
COUNT(CASE WHEN strftime('%w',pickup_datetime) IS '1' THEN 1 END) AS 'Monday',
COUNT(CASE WHEN strftime('%w',pickup_datetime) IS '2' THEN 1 END) AS 'Tuesday',
COUNT(CASE WHEN strftime('%w',pickup_datetime) IS '3' THEN 1 END) AS 'Wednesday',
COUNT(CASE WHEN strftime('%w',pickup_datetime) IS '4' THEN 1 END) AS 'Thursday',
COUNT(CASE WHEN strftime('%w',pickup_datetime) IS '5' THEN 1 END) AS 'Friday',
COUNT(CASE WHEN strftime('%w',pickup_datetime) IS '6' THEN 1 END) AS 'Saturday'
FROM uber
"""

In [32]:
engine.execute(QUERY_2).fetchall()

[(25834, 24681, 27526, 28328, 29338, 30166, 29599)]

In [33]:
write_query_to_file(QUERY_2, "popular_uber_days.sql")

### Query 3

Here we will find the 95% percentile of distance traveled for all hired trips during July 2013 by forming a union between our Yellow Taxi data and Uber data and finding what distance encompasses 95% of all rides.


In [34]:
QUERY_3 = """
SELECT distance
FROM taxi

UNION ALL

SELECT distance
FROM uber
ORDER BY distance DESC
LIMIT 1 OFFSET 19055
"""

In [35]:
engine.execute(QUERY_3).fetchall()

[(10.438645448958848,)]

In [36]:
write_query_to_file(QUERY_3, "rides_95th_percentile.sql")

### Query 4

Here we find the 10 days with the most hired rides and their average distance by filtering our union and grouping by day. We then order by number of rides per day and select the top 10 days.

In [37]:
QUERY_4 = """
SELECT pickup_datetime, AVG(distance)
FROM (
SELECT *
FROM taxi
WHERE pickup_datetime LIKE '2009%'

UNION ALL

SELECT *
FROM uber
WHERE pickup_datetime LIKE '2009%'
)
GROUP BY pickup_datetime
ORDER BY COUNT(pickup_datetime) DESC
LIMIT 10
"""

In [38]:
engine.execute(QUERY_4).fetchall()

[('2009-08-21 19:01:00.000000', 2.3004645356549798),
 ('2009-02-12 12:46:00.000000', 2.06106123864422),
 ('2009-12-29 19:49:00.000000', 1.359449274955546),
 ('2009-12-22 11:08:00.000000', 1.0413531499025293),
 ('2009-12-16 13:05:00.000000', 2.591184118342559),
 ('2009-12-06 18:24:00.000000', 3.0824503260395963),
 ('2009-12-01 15:03:00.000000', 2.170797180883661),
 ('2009-11-23 17:51:00.000000', 2.4266183086888327),
 ('2009-11-14 00:41:00.000000', 7.162453247895702),
 ('2009-11-05 23:39:00.000000', 3.015314787749366)]

In [39]:
write_query_to_file(QUERY_4, "avg_dist_of_busiest_days_2009.sql")

### Query 5

Here we will find what days in 2014 were the windiest and how many rides were taken on those days by grouping our rides by the day, joining with our daily weather reports, and ordering by windiest days.


In [69]:
QUERY_5 = """
SELECT strftime('%Y-%m-%d',pickup_datetime) AS date, COUNT(
    (SELECT distance FROM taxi WHERE pickup_datetime LIKE '2014%'
    UNION ALL
    SELECT distance FROM uber WHERE pickup_datetime LIKE '2014%'
    ) 
    )
FROM (
SELECT *
FROM taxi
WHERE pickup_datetime LIKE '2014%'

UNION ALL

SELECT *
FROM uber
WHERE pickup_datetime LIKE '2014%'
) AS a

JOIN dailyWeather ON strftime('%Y-%m-%d',a.pickup_datetime)=dailyWeather.id
GROUP BY strftime('%Y-%m-%d',pickup_datetime)
ORDER BY dailyWeather.DailyWindSpeed DESC
LIMIT 10
"""

In [70]:
engine.execute(QUERY_5).fetchall()

[('2014-03-13', 197),
 ('2014-01-07', 162),
 ('2014-01-02', 125),
 ('2014-02-13', 119),
 ('2014-03-26', 170),
 ('2014-03-29', 199),
 ('2014-12-07', 156),
 ('2014-12-09', 153),
 ('2014-12-08', 154),
 ('2014-11-02', 147)]

In [71]:
write_query_to_file(QUERY_5, "windy_days_with_trips.sql")

### Query 6

Here we will find the number of rides taken, precipitation and wind speed for every hour in the two week period surrounding Hurricane Sandy's presence in NYC. We group our rides by hour and limit our search to the desired time period.


In [95]:
QUERY_6 = """
SELECT COUNT(
    (SELECT distance FROM taxi WHERE (
       pickup_datetime LIKE '2012-10-22%'
    OR pickup_datetime LIKE '2012-10-23%'
    OR pickup_datetime LIKE '2012-10-24%'
    OR pickup_datetime LIKE '2012-10-25%'
    OR pickup_datetime LIKE '2012-10-26%'
    OR pickup_datetime LIKE '2012-10-27%'
    OR pickup_datetime LIKE '2012-10-28%'
    OR pickup_datetime LIKE '2012-10-29%'
    OR pickup_datetime LIKE '2012-10-30%'
    OR pickup_datetime LIKE '2012-10-31%'
    OR pickup_datetime LIKE '2012-11-01%'
    OR pickup_datetime LIKE '2012-11-02%'
    OR pickup_datetime LIKE '2012-11-03%'
    OR pickup_datetime LIKE '2012-11-04%'
    OR pickup_datetime LIKE '2012-11-05%'
    OR pickup_datetime LIKE '2012-11-06%'
    )
    
    UNION ALL
    
    SELECT distance FROM uber WHERE (
       pickup_datetime LIKE '2012-10-22%'
    OR pickup_datetime LIKE '2012-10-23%'
    OR pickup_datetime LIKE '2012-10-24%'
    OR pickup_datetime LIKE '2012-10-25%'
    OR pickup_datetime LIKE '2012-10-26%'
    OR pickup_datetime LIKE '2012-10-27%'
    OR pickup_datetime LIKE '2012-10-28%'
    OR pickup_datetime LIKE '2012-10-29%'
    OR pickup_datetime LIKE '2012-10-30%'
    OR pickup_datetime LIKE '2012-10-31%'
    OR pickup_datetime LIKE '2012-11-01%'
    OR pickup_datetime LIKE '2012-11-02%'
    OR pickup_datetime LIKE '2012-11-03%'
    OR pickup_datetime LIKE '2012-11-04%'
    OR pickup_datetime LIKE '2012-11-05%'
    OR pickup_datetime LIKE '2012-11-06%'
    ) 
    )), hourlyWeather.HourlyPrecipitation, hourlyWeather.HourlyWindSpeed
FROM (
    SELECT * FROM taxi WHERE (
       pickup_datetime LIKE '2012-10-22%'
    OR pickup_datetime LIKE '2012-10-23%'
    OR pickup_datetime LIKE '2012-10-24%'
    OR pickup_datetime LIKE '2012-10-25%'
    OR pickup_datetime LIKE '2012-10-26%'
    OR pickup_datetime LIKE '2012-10-27%'
    OR pickup_datetime LIKE '2012-10-28%'
    OR pickup_datetime LIKE '2012-10-29%'
    OR pickup_datetime LIKE '2012-10-30%'
    OR pickup_datetime LIKE '2012-10-31%'
    OR pickup_datetime LIKE '2012-11-01%'
    OR pickup_datetime LIKE '2012-11-02%'
    OR pickup_datetime LIKE '2012-11-03%'
    OR pickup_datetime LIKE '2012-11-04%'
    OR pickup_datetime LIKE '2012-11-05%'
    OR pickup_datetime LIKE '2012-11-06%'
    )
    
    UNION ALL
    
    SELECT * FROM uber WHERE (
       pickup_datetime LIKE '2012-10-22%'
    OR pickup_datetime LIKE '2012-10-23%'
    OR pickup_datetime LIKE '2012-10-24%'
    OR pickup_datetime LIKE '2012-10-25%'
    OR pickup_datetime LIKE '2012-10-26%'
    OR pickup_datetime LIKE '2012-10-27%'
    OR pickup_datetime LIKE '2012-10-28%'
    OR pickup_datetime LIKE '2012-10-29%'
    OR pickup_datetime LIKE '2012-10-30%'
    OR pickup_datetime LIKE '2012-10-31%'
    OR pickup_datetime LIKE '2012-11-01%'
    OR pickup_datetime LIKE '2012-11-02%'
    OR pickup_datetime LIKE '2012-11-03%'
    OR pickup_datetime LIKE '2012-11-04%'
    OR pickup_datetime LIKE '2012-11-05%'
    OR pickup_datetime LIKE '2012-11-06%'
    )
) AS a
JOIN hourlyWeather ON strftime('%Y-%m-%d %H',a.pickup_datetime)=strftime('%Y-%m-%d %H',hourlyWeather.date)
GROUP BY strftime('%Y-%m-%d %H',a.pickup_datetime)
"""

In [96]:
engine.execute(QUERY_6).fetchall()

[(2, 0.0, 7.0),
 (2, 0.0, 5.0),
 (2, 0.0, 7.0),
 (1, 0.0, 0.0),
 (1, 0.0, 0.0),
 (6, 0.0, 5.0),
 (10, 0.0, 3.0),
 (8, 0.0, 3.0),
 (9, 0.0, 5.0),
 (8, 0.0, 11.0),
 (7, 0.0, 7.0),
 (7, 0.0, 6.0),
 (8, 0.0, 3.0),
 (8, 0.0, 7.0),
 (15, 0.0, 5.0),
 (4, 0.0, 5.0),
 (11, 0.0, 3.0),
 (5, 0.0, 0.0),
 (10, 0.0, 3.0),
 (5, 0.0, 3.0),
 (12, 0.02, 0.0),
 (11, 0.01, 0.0),
 (2, 0.0, 3.0),
 (1, 0.0, 5.0),
 (4, 0.0, 5.0),
 (1, 0.0, 7.0),
 (1, 0.0, 7.0),
 (5, 0.0, 6.0),
 (1, 0.0, 3.0),
 (4, 0.0, 7.0),
 (9, 0.0, 7.0),
 (7, 0.0, 8.0),
 (3, 0.0, 7.0),
 (9, 0.0, 7.0),
 (5, 0.0, 8.0),
 (8, 0.0, 5.0),
 (6, 0.0, 7.0),
 (14, 0.0, 8.0),
 (17, 0.0, 3.0),
 (12, 0.0, 5.0),
 (7, 0.0, 6.0),
 (9, 0.0, 0.0),
 (8, 0.0, 3.0),
 (3, 0.0, 3.0),
 (2, 0.0, 3.0),
 (1, 0.0, 6.0),
 (3, 0.0, 0.0),
 (7, 0.0, 5.0),
 (9, 0.0, 6.0),
 (9, 0.0, 5.0),
 (7, 0.0, 3.0),
 (9, 0.0, 6.0),
 (13, 0.0, 0.0),
 (10, 0.0, 6.0),
 (10, 0.0, 0.0),
 (11, 0.0, 5.0),
 (7, 0.0, 5.0),
 (4, 0.0, 0.0),
 (8, 0.0, 3.0),
 (12, 0.0, 0.0),
 (4, 0.0, 0.0),
 (14, 0

In [97]:
write_query_to_file(QUERY_6, "hurricane_rides_and_weather_by_hour.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_n():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplemented()

In [ ]:
some_dataframe = get_data_for_visual_n()
plot_visual_n(some_dataframe)